# Python Algorithmic Trading Cookbook

## Chapter 8: Algorithmic Trading: Code Strategies Step-by-step

This Jupyter Notebook is created using Python version 3.8.2

----

### Master Recipe

The following code will help you set up the broker connection with Zerodha, which will be used by all the recipes in this chapter. Please make sure you have followed these steps before trying out any recipe. 


In [1]:
from pyalgotrading.strategy.strategy_base import StrategyBase
from pyalgotrading.constants import *

----

### Recipe 1: EMA-Regular-Order Strategy: Coding the '__init__', '\_\_initialize\_\_', 'name' and 'versions_supported' methods

In [2]:
class StrategyEMARegularOrder(StrategyBase):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.timeperiod1 = self.strategy_parameters['timeperiod1']
        self.timeperiod2 = self.strategy_parameters['timeperiod2']

        self.main_order = None

    def initialize(self):
        self.main_order = {}
    
    @staticmethod
    def name():
        return 'EMA Regular Order Strategy'

    @staticmethod    
    def versions_supported():
        return AlgoBullsEngineVersion.VERSION_3_2_0

### Recipe 2: EMA-Regular-Order Strategy: Coding the 'strategy_select_instruments_for_entry' method

In [3]:
class StrategyEMARegularOrder(StrategyBase):
   
    # Previous methods not shown

    def get_crossover_value(self, instrument):
        hist_data = self.get_historical_data(instrument)
        ema_x = talib.EMA(hist_data['close'], timeperiod=self.timeperiod1)
        ema_y = talib.EMA(hist_data['close'], timeperiod=self.timeperiod2)
        crossover_value = self.utils.crossover(ema_x, ema_y)
        return crossover_value

    def strategy_select_instruments_for_entry(self, candle, instruments_bucket):

        selected_instruments_bucket = []
        sideband_info_bucket = []

        for instrument in instruments_bucket:
            crossover_value = self.get_crossover_value(instrument)
            if crossover_value == 1:
                selected_instruments_bucket.append(instrument)
                sideband_info_bucket.append({'action': 'BUY'})
            elif crossover_value == -1:
                if self.strategy_mode is StrategyMode.INTRADAY:
                    selected_instruments_bucket.append(instrument)
                    sideband_info_bucket.append({'action': 'SELL'})
                    
        return selected_instruments_bucket, sideband_info_bucket

### Recipe 3: EMA-Regular-Order Strategy: Coding the 'strategy_enter_position' method

In [4]:
class StrategyEMARegularOrder(StrategyBase):
   
    # Previous methods not shown

    def strategy_enter_position(self, candle, instrument, sideband_info):
        if sideband_info['action'] == 'BUY':
            qty = self.number_of_lots * instrument.lot_size
            self.main_order[instrument] = \
                self.broker.BuyOrderRegular(instrument=instrument,
                                            order_code=BrokerOrderCodeConstants.INTRADAY,
                                            order_variety=BrokerOrderVarietyConstants.MARKET,
                                            quantity=qty)
        elif sideband_info['action'] == 'SELL':
            qty = self.number_of_lots * instrument.lot_size
            self.main_order[instrument] = \
                self.broker.SellOrderRegular(instrument=instrument,
                                             order_code=BrokerOrderCodeConstants.INTRADAY,
                                             order_variety=BrokerOrderVarietyConstants.MARKET,
                                             quantity=qty)
        else:
            raise SystemExit(f'Got invalid sideband_info value: {sideband_info}')

        return self.main_order[instrument]

### Recipe 4: EMA-Regular-Order Strategy: Coding the 'strategy_select_instruments_for_exit' method

In [5]:
class StrategyEMARegularOrder(StrategyBase):
   
    # Previous methods not shown

    def strategy_select_instruments_for_exit(self, candle, instruments_bucket):
        selected_instruments_bucket = []
        sideband_info_bucket = []

        for instrument in instruments_bucket:
            if self.main_order.get(instrument) is not None:
                crossover_value = self.get_crossover_value(instrument)
                if crossover_value in [1, -1]:
                    selected_instruments_bucket.append(instrument)
                    sideband_info_bucket.append({'action': 'EXIT'})
        return selected_instruments_bucket, sideband_info_bucket

### Recipe 5: EMA-Regular-Order Strategy: Coding the 'strategy_exit_position' method

In [6]:
class StrategyEMARegularOrder(StrategyBase):
   
    # Previous methods not shown

    def strategy_exit_position(self, candle, instrument, sideband_info):
        if sideband_info['action'] == 'EXIT':
            self.main_order[instrument].exit_position()
            self.main_order[instrument] = None
            return True

        return False

### Recipe 6:  EMA-Regular-Order Strategy: Create strategy on AlgoBulls Trading Platform

In [7]:
import inspect
from pyalgotrading.algobulls import AlgoBullsConnection
from pyalgostrategypool.strategy_ema_regular_order import StrategyEMARegularOrder

In [8]:
algobulls_connection = AlgoBullsConnection()

In [9]:
algobulls_connection.get_authorization_url()

Please login to this URL with your AlgoBulls credentials and get your developer access token: https://app.algobulls.com/user/login


'https://app.algobulls.com/user/login'

In [10]:
algobulls_connection.set_access_token('80b7a69b168c5b3f15d56688841a8f2da5e2ab2c')

In [11]:
print(inspect.getsource(StrategyEMARegularOrder))

class StrategyEMARegularOrder(StrategyBase):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.timeperiod1 = self.strategy_parameters['timeperiod1']
        self.timeperiod2 = self.strategy_parameters['timeperiod2']

        self.main_order = None

    def initialize(self):
        self.main_order = {}

    @staticmethod
    def name():
        return 'EMA Regular Order Strategy'

    @staticmethod
    def versions_supported():
        return AlgoBullsEngineVersion.VERSION_3_2_0

    def get_crossover_value(self, instrument):
        hist_data = self.get_historical_data(instrument)
        ema_x = talib.EMA(hist_data['close'], timeperiod=self.timeperiod1)
        ema_y = talib.EMA(hist_data['close'], timeperiod=self.timeperiod2)
        crossover_value = self.utils.crossover(ema_x, ema_y)
        return crossover_value

    def strategy_select_instruments_for_entry(self, candle, instruments_bucket):

        selected_instruments_bucket = 

In [12]:
algobulls_connection.create_strategy(StrategyEMARegularOrder)

Validating Strategy...


{'data': 'success', 'strategyCode': '49287246f9704bbcbad76ade9e2091d9'}

### There's more...

In [13]:
algobulls_connection.create_strategy(StrategyEMARegularOrder, overwrite=True)

Validating Strategy...


{'data': 'success'}

### Recipe 7: MACD-Bracket-Order Strategy: Coding the '__init__', '\_\_initialize\_\_', 'name' and 'versions_supported' methods

In [14]:
class StrategyMACDBracketOrder(StrategyBase):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.fastMA_period = self.strategy_parameters['fastma_period']
        self.slowMA_period = self.strategy_parameters['slowma_period']
        self.signal_period = self.strategy_parameters['signal_period']
        self.stoploss = self.strategy_parameters['stoploss_trigger']
        self.target = self.strategy_parameters['target_trigger']
        self.trailing_stoploss = self.strategy_parameters['trailing_stoploss_trigger']

        self.main_order = None

    def initialize(self):
        self.main_order = {}

    @staticmethod
    def name():
        return 'MACD Bracket Order Strategy'

    @staticmethod
    def versions_supported():
        return VERSION_3_2_0

### Recipe 8: MACD-Bracket-Order Strategy: Coding the 'strategy_select_instruments_for_entry' method

In [15]:
class StrategyMACDBracketOrder(StrategyBase):
    
    # Note: Some methods are not shown here    
    
    def get_crossover_value(self, instrument):
        hist_data = self.get_historical_data(instrument)
        macdline, macdsignal, _ = talib.MACD(hist_data['close'], 
                                             fastperiod=self.fastMA_period, 
                                             slowperiod=self.slowMA_period, 
                                             signalperiod=self.signal_period)
        crossover_value = crossover(macdline, macdsignal)
        return crossover_value

    def strategy_select_instruments_for_entry(self, candle, instruments_bucket):

        selected_instruments_bucket = []
        sideband_info_bucket = []

        for instrument in instruments_bucket:
            crossover_value = self.get_crossover_value(instrument)
            if crossover_value == 1:
                selected_instruments_bucket.append(instrument)
                sideband_info_bucket.append({'action': 'BUY'})
            elif crossover_value == -1:
                if self.strategy_mode is StrategyMode.INTRADAY:
                    selected_instruments_bucket.append(instrument)
                    sideband_info_bucket.append({'action': 'SELL'})

        return selected_instruments_bucket, sideband_info_bucket    

### Recipe 9: MACD-Bracket-Order Strategy: Coding the 'strategy_enter_position' method

In [16]:
class StrategyMACDBracketOrder(StrategyBase):
    
    # Note: Some methods are not shown here 
    
    def strategy_enter_position(self, candle, instrument, sideband_info):
        if sideband_info['action'] == 'BUY':
            qty = self.number_of_lots * instrument.lot_size
            ltp = self.broker.get_ltp(instrument)
            self.main_order[instrument] = \
                self.broker.BuyOrderBracket(instrument=instrument,
                                            order_code=BrokerOrderCodeConstants.INTRADAY,
                                            order_variety=BrokerOrderVarietyConstants.LIMIT,
                                            quantity=qty,
                                            price=ltp,
                                            stoploss_trigger=ltp - (ltp * self.stoploss),
                                            target_trigger=ltp + (ltp * self.target),
                                            trailing_stoploss_trigger=ltp * self.trailing_stoploss)

        elif sideband_info['action'] == 'SELL':
            qty = self.number_of_lots * instrument.lot_size
            ltp = self.broker.get_ltp(instrument)
            self.main_order[instrument] = \
                self.broker.SellOrderBracket(instrument=instrument,
                                             order_code=BrokerOrderCodeConstants.INTRADAY,
                                             order_variety=BrokerOrderVarietyConstants.LIMIT,
                                             quantity=qty,
                                             price=ltp,
                                             stoploss_trigger=ltp + (ltp * self.stoploss),
                                             target_trigger=ltp - (ltp * self.target),
                                             trailing_stoploss_trigger=ltp * self.trailing_stoploss)
        else:
            raise SystemExit(f'Got invalid sideband_info value: {sideband_info}')

        return self.main_order[instrument]
    

### Recipe 10: MACD-Bracket-Order Strategy: Coding the 'strategy_select_instruments_for_exit' method

In [17]:
class StrategyMACDBracketOrder(StrategyBase):
    
    # Note: Some methods are not shown here   
    
    def strategy_select_instruments_for_exit(self, candle, instruments_bucket):
        selected_instruments_bucket = []
        sideband_info_bucket = []

        for instrument in instruments_bucket:
            if self.main_order.get(instrument) is not None:
                crossover_value = self.get_crossover_value(instrument)
                if crossover_value in [1, -1]:
                    selected_instruments_bucket.append(instrument)
                    sideband_info_bucket.append({'action': 'EXIT'})
        return selected_instruments_bucket, sideband_info_bucket    

### Recipe 11: MACD-Bracket-Order Strategy: Coding the 'strategy_exit_position' method

In [18]:
class StrategyMACDBracketOrder(StrategyBase):
    
    # Note: Some methods are not shown here   
    
    def strategy_exit_position(self, candle, instrument, sideband_info):
        if sideband_info['action'] == 'EXIT':
            self.main_order[instrument].exit_position()
            self.main_order[instrument] = None
            return True

        return False

### Recipe 12: MACD-Bracket-Order Strategy: Create strategy on AlgoBulls Trading Platform

In [19]:
import inspect
from pyalgostrategypool.strategy_macd_bracket_order import StrategyMACDBracketOrder
from pyalgotrading.algobulls import AlgoBullsConnection

In [20]:
algobulls_connection = AlgoBullsConnection()

In [21]:
algobulls_connection.get_authorization_url()

Please login to this URL with your AlgoBulls credentials and get your developer access token: https://app.algobulls.com/user/login


'https://app.algobulls.com/user/login'

In [22]:
algobulls_connection.set_access_token('80b7a69b168c5b3f15d56688841a8f2da5e2ab2c')

In [23]:
print(inspect.getsource(StrategyMACDBracketOrder))

class StrategyMACDBracketOrder(StrategyBase):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.fastMA_period = self.strategy_parameters['fastma_period']
        self.slowMA_period = self.strategy_parameters['slowma_period']
        self.signal_period = self.strategy_parameters['signal_period']
        self.stoploss = self.strategy_parameters['stoploss_trigger']
        self.target = self.strategy_parameters['target_trigger']
        self.trailing_stoploss = self.strategy_parameters['trailing_stoploss_trigger']

        self.main_order = None

    def initialize(self):
        self.main_order = {}

    @staticmethod
    def name():
        return 'MACD Bracket Order Strategy'

    @staticmethod
    def versions_supported():
        return AlgoBullsEngineVersion.VERSION_3_2_0

    def get_crossover_value(self, instrument):
        hist_data = self.get_historical_data(instrument)
        macdline, macdsignal, _ = talib.MACD(hist_data['close

In [24]:
algobulls_connection.create_strategy(StrategyMACDBracketOrder)

Validating Strategy...


{'data': 'success', 'strategyCode': '4faf514fe096432b8e9f80f5951bd2ea'}

### There's more...

In [25]:
algobulls_connection.create_strategy(StrategyMACDBracketOrder, overwrite=True)

Validating Strategy...


{'data': 'success'}